In [1]:
!pip install libigl matplotlib numpy polyscope gpytoolbox

     |████████████████████████████████| 14.4 MB 20 kB/s  eta 0:00:01     |██████████████████████████████▉ | 13.8 MB 5.6 MB/s eta 0:00:01
     |████████████████████████████████| 3.5 MB 8.4 MB/s eta 0:00:01
     |████████████████████████████████| 9.5 MB 7.0 MB/s eta 0:00:01


In [3]:
## Testing polyscope works in jupyter server 
import polyscope as ps 
import numpy as np

ps.init()
ps.remove_all_structures()
ps.register_surface_mesh("single tri", np.array([[0,0,0],[0,1,0],[0,0,1]]), np.array([[0,1,2]]), edge_width=1)
ps.show()

## Mesh Parameterization Intro Pt. 1
In this notebook we will

In [ ]:
### Example 1: Single 3D Triangle ###

# Define the triangle and visualize

In [ ]:
# Every possible parameterization of the triangle can be represented by a 3x2 linear transformation and corresponding 2-D translation

# Let's show this by assigning arbitrary 2D points to the triangle vertices and finding the linear transformation (i.e. Jacobian) which corresponds with them

In [ ]:
# There's another way to define a parameterization: project the points onto the subspace defined by any two triangle edge vectors
# We will refer to this as the trivial parameterization (in which each triangle is projected onto its local basis to define an isometric embedding)

In [ ]:
# Let's confirm the trivial parameterization is distortionless by computing its singular values

In [ ]:
### Example 2: Open Pyramid ###
# We will now see the effect of different choices of barycentric coordinates (spring constants) on the resulting parameterization